In [2]:
import pandas as pd
import sqlite3
import dbhelper
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pytz

In [3]:
cityname = 'Schöneberg'
tz = pytz.timezone('Europe/Berlin')
print(tz)

Europe/Berlin


In [4]:
df_orig = pd.read_csv('auseec\sbrg.csv',sep=';').drop(['Unnamed: 0'],axis=1)
df_orig.columns= ['timestampUNX','city','country','sunrise','sunset','temperature','humidity','cloudiness','pressure','windspeed','winddeg']
df_orig = df_orig[['country','city','timestampUNX','sunrise','sunset','temperature','humidity','pressure','windspeed','winddeg','cloudiness']]

In [5]:
df_time = df_orig.copy()
df_time['timestamp'] = df_time['timestampUNX'].apply(lambda x: dt.datetime.strptime(x,"%d.%m.%y %H:%M"))
df_time['timezone'] = df_time['timestampUNX'].apply(lambda x:  int(int(dt.datetime.strptime(x,"%d.%m.%y %H:%M").astimezone(tz).strftime('%z'))/100))
df_time['timestampUNX'] = df_time.apply(lambda x: x['timestamp'].timestamp() - x['timezone']*3600, axis  =1)

df_time['sunrise'] = df_time.apply(lambda x: x['timestamp'].strftime("%d.%m.%y") + ' ' + x['sunrise'], axis = 1)
df_time['sunrise'] = df_time.apply(lambda x: dt.datetime.strptime(x['sunrise'],"%d.%m.%y %H:%M").timestamp() - x['timezone']*3600, axis = 1)

df_time['sunset'] = df_time.apply(lambda x: x['timestamp'].strftime("%d.%m.%y") + ' ' + x['sunset'], axis = 1)
df_time['sunset'] = df_time.apply(lambda x: dt.datetime.strptime(x['sunset'],"%d.%m.%y %H:%M").timestamp() - x['timezone']*3600, axis = 1)

print(df_time)

      country                city  timestampUNX       sunrise        sunset  \
0          DE  Berlin Schoeneberg  1.539332e+09  1.539315e+09  1.539354e+09   
1          DE  Berlin Schoeneberg  1.539334e+09  1.539315e+09  1.539354e+09   
2          DE  Berlin Schoeneberg  1.539336e+09  1.539315e+09  1.539354e+09   
3          DE  Berlin Schoeneberg  1.539338e+09  1.539315e+09  1.539354e+09   
4          DE  Berlin Schoeneberg  1.539340e+09  1.539315e+09  1.539354e+09   
...       ...                 ...           ...           ...           ...   
43117      DE  Berlin Schoeneberg  1.579786e+09  1.579759e+09  1.579790e+09   
43118      DE  Berlin Schoeneberg  1.579787e+09  1.579759e+09  1.579790e+09   
43119      DE  Berlin Schoeneberg  1.579788e+09  1.579759e+09  1.579790e+09   
43120      DE  Berlin Schoeneberg  1.579788e+09  1.579759e+09  1.579790e+09   
43121      DE  Berlin Schoeneberg  1.579789e+09  1.579759e+09  1.579790e+09   

      temperature  humidity  pressure windspeed  wi

In [6]:
df = df_time.copy()
df['temperature'] = df['temperature'].apply( lambda x: float(x.replace(',','.')) )
df['windspeed'] = df['windspeed'].apply( lambda x: float(x.replace(',','.')) )
df['winddeg'] = df['winddeg'].apply( lambda x: str(x).replace('.0','') )
df['winddeg'] = df['winddeg'].apply( lambda x: x.replace('nan',''))
df = df[['country','city','timestampUNX','sunrise','sunset','temperature','humidity','pressure','windspeed','winddeg','cloudiness','timezone']]
print(df)


      country                city  timestampUNX       sunrise        sunset  \
0          DE  Berlin Schoeneberg  1.539332e+09  1.539315e+09  1.539354e+09   
1          DE  Berlin Schoeneberg  1.539334e+09  1.539315e+09  1.539354e+09   
2          DE  Berlin Schoeneberg  1.539336e+09  1.539315e+09  1.539354e+09   
3          DE  Berlin Schoeneberg  1.539338e+09  1.539315e+09  1.539354e+09   
4          DE  Berlin Schoeneberg  1.539340e+09  1.539315e+09  1.539354e+09   
...       ...                 ...           ...           ...           ...   
43117      DE  Berlin Schoeneberg  1.579786e+09  1.579759e+09  1.579790e+09   
43118      DE  Berlin Schoeneberg  1.579787e+09  1.579759e+09  1.579790e+09   
43119      DE  Berlin Schoeneberg  1.579788e+09  1.579759e+09  1.579790e+09   
43120      DE  Berlin Schoeneberg  1.579788e+09  1.579759e+09  1.579790e+09   
43121      DE  Berlin Schoeneberg  1.579789e+09  1.579759e+09  1.579790e+09   

       temperature  humidity  pressure  windspeed w

In [12]:
cityID = dbhelper.execComFK('''SELECT ctyid FROM Cities WHERE cityname = '{}' '''.format(cityname))[0][0]
firstAvail = dbhelper.execComFK('''SELECT MIN(timestampUNX) FROM Wetter WHERE ctyid = '{}' '''.format(cityID))[0][0]
print('cityID: {}, first: {}'.format(cityID,firstAvail))

cityID: 1, first: 1582811015


In [14]:
def insertRow(wth, ctyid):    
    try:
        sqliteConnection = sqlite3.connect('wetter.db')
        cursor = sqliteConnection.cursor()
        # 'country','city','timestampUNX','sunrise','sunset','temperature','humidity','pressure','windspeed','winddeg','cloudiness','timezone',ctyid
        sqlite_insert_query = """INSERT INTO Wetter
                                (country,city,timestampUNX,sunrise,sunset,temperature,humidity,pressure,windspeed,winddeg,cloudiness,timezone,ctyid) 
                                VALUES 
                                (?,?,?,?,?,?,?,?,?,?,?,?,?)"""
        data_tuple = (
                        wth['country'], wth['city'], wth['timestampUNX'],
                        wth['sunrise'], wth['sunset'], wth['temperature'],
                        wth['humidity'], wth['pressure'], wth['windspeed'], wth['winddeg'],
                        wth['cloudiness'], wth['timezone'], ctyid
                        ) 

        count = cursor.execute(sqlite_insert_query, data_tuple)
        sqliteConnection.commit()
        
        cursor.close()
    except sqlite3.Error as error:
        print("Error while inserted into Wetter table City {}, Error: {}".format(wth.city,error))

rows = df.count(axis=0)['country']
print(rows)


43122


In [15]:
import os
for idx, row in df.iterrows():
    if row['timestampUNX'] < firstAvail:
        insertRow(row,cityID)
    os.system('cls')
    print('{}'.format(float(idx)/float(rows)))

0.0
2.3190019015815594e-05
4.638003803163119e-05
6.957005704744678e-05
9.276007606326238e-05
0.00011595009507907797
0.00013914011409489355
0.00016233013311070914
0.00018552015212652476
0.00020871017114234034
0.00023190019015815593
0.0002550902091739715
0.0002782802281897871
0.0003014702472056027
0.0003246602662214183
0.00034785028523723387
0.0003710403042530495
0.0003942303232688651
0.0004174203422846807
0.0004406103613004963
0.00046380038031631186
0.00048699039933212745
0.000510180418347943
0.0005333704373637586
0.0005565604563795742
0.0005797504753953898
0.0006029404944112054
0.000626130513427021
0.0006493205324428366
0.0006725105514586521
0.0006957005704744677
0.0007188905894902834
0.000742080608506099
0.0007652706275219146
0.0007884606465377302
0.0008116506655535458
0.0008348406845693614
0.000858030703585177
0.0008812207226009925
0.0009044107416168081
0.0009276007606326237
0.0009507907796484393
0.0009739807986642549
0.0009971708176800705
0.001020360836695886
0.0010435508557117017
0